In [ ]:
def decoder(x,shift):
    chars = """!"#$%\'()*+,-./:;<=>?_¡£¥¿&¤0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyzÄÅÆÇÉÑØøÜßÖàäåæèéìñòöùüΔΦΓΛΩΠΨΣΘΞ"""
    extrachars = {'@!':122,
    '@@':123,
    '@#':124,
    '@$':125,
    '@%':126,
    '@?':127}
    new_chars = chars[shift:] + chars[:shift]
    
    decoded = []
    counter = 0
    while(counter < len(x)):
        if x[counter] == '@':
            decoded.append("{0:07b}".format(extrachars[x[counter:counter+2]]))
            counter = counter + 2
        else:
            decoded.append("{0:07b}".format(new_chars.index(x[counter])))
            counter = counter + 1
    
    decoded = ''.join(decoded)
    decoded = [decoded[i:i+4] for i in range(0, len(decoded), 4)]
    if len(decoded[-1]) < 4:
        del decoded[-1]
    decoded = [int(i,2) for i in decoded]
    mag = pd.Series(decoded[:int(len(decoded)/2)])
    dirs = pd.Series(decoded[int(len(decoded)/2):]).reset_index(drop=True)
    
    mag = mag*5
    v10 = np.sin(2*np.pi*dirs/16)*mag
    u10 = np.cos(2*np.pi*dirs/16)*mag
    
    idx = []
    for h in hours:
        for lat in np.arange(float(maxmin[0]),float(maxmin[1]) + float(interval[0]),float(interval[0])):
            for lon in np.arange(float(maxmin[2]),float(maxmin[3]) + float(interval[1]),float(interval[1])):
                idx.append({'hour':h,'lat':lat,'lon':lon})
    decoded = pd.DataFrame(idx)
    decoded['u10'] = u10
    decoded['v10'] = v10
    
    return decoded.set_index(['hour','lat','lon'])

In [ ]:
import pandas as pd
import numpy as np
FILE_LOCATION = ''
with open(FILE_LOCATION, encoding='utf-8') as f:
    received = f.read()
hours = received.split('\n')[0].split(',')
gribtime = received.split('\n')[1]
maxmin = received.split('\n')[2].split(',')
interval = received.split('\n')[3].split(',')
shift = received.split('\n')[4]
df = decoder(''.join(received.split('\n')[5:-2][::3]),1)

In [ ]:
%matplotlib notebook
#import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import geopandas
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
OUR_LOCATION = (-62.93,20.8219)
WORLD_COUNTRIES_FILE_LOCATION = '' # If you want a more high resolution map
#world = geopandas.read_file("WORLD_COUNTRIES_FILE_LOCATION")
# world = world[world['CNTRY_NAME'].isin(['Anguilla', 'Antigua & Barbuda', 'Barbados', 'Belize', 'Bermuda',
#        'Canada', 'Cayman Is.', 'Clipperton Island (France)', 'Colombia',
#        'Costa Rica', 'Cuba', 'Dominica', 'Dominican Republic',
#        'El Salvador', 'Greenland', 'Grenada', 'Guadeloupe', 'Guatemala',
#        'Haiti', 'Honduras', 'In dispute Belize/Honduras',
#        'In dispute Canada/Denmark', 'In dispute El Salvador/Honduras',
#        'Jamaica', 'Martinique', 'Mexico', 'Montserrat',
#        'Navassa Island (US)', 'Nicaragua', 'Jarvis I.', 'Panama',
#        'Puerto Rico', 'Saba (Neth)', 'Saint Barthelemy (France)',
#        'St. Kitts & Nevis', 'St. Lucia', 'Saint Martin (France)',
#        'St. Pierre & Miquelon', 'St. Vincent & the Grenadines',
#        'Sint Eustatius (Neth)', 'Sint Maarten (Neth)', 'The Bahamas',
#        'Turks & Caicos Is.', 'United States', 'British Virgin Is.',
#        'Virgin Is.'])]

for timepoint in df.index.get_level_values(0).unique():
    g = df.loc[timepoint].reset_index()
    #fig = plt.figure()
    
    #fig = plt.figure(figsize=(15,10))
    world.plot(figsize=(8,5))
    #ax = fig.add_subplot(1, 1, 1, projection=ccrs.LambertConformal())
    plt.barbs(g['lon'], g['lat'], g['u10'], g['v10'], length=6)
    plt.plot(OUR_LOCATION[0], OUR_LOCATION[1], marker='D',c='red')
    #plt.plot(-(63+0/60), 27+40/60, marker='D',c='red')

    plt.xlim([-64.5,-60])
    plt.ylim([17.5,22])
    plt.title((pd.to_datetime(gribtime) + pd.Timedelta(hours=int(timepoint))).tz_localize('UTC').tz_convert('US/Eastern'))
    #plt.show()
    plt.grid(True)
    #, np.sqrt(g['u10']**2 + g['v10']**2), cmap='rainbow'